# Working with spatial data

Notebook developed by Sam Maurer

This demo introduces some basic geospatial analysis operations in Python. We'll look at distance calculations, loading geospatial datasets, what the data types look like, and how to perform spatial joins. Some helpful tools:

#### GeoPandas
- extension to Pandas to support operations with geospatial data
- https://geopandas.org

#### Shapely
- low-level spatial data types and processing algorithms
- https://shapely.readthedocs.io/en/latest/manual.html

#### Fiona
- reading spatial data files (other libraries use this behind the scenes)
- https://fiona.readthedocs.io/

#### GeoPy
- distance algorithms and automated geocoding
- https://geopy.readthedocs.io/

## 1. Calculating distances

You can perform geospatial analysis in Python even without starting from an existing dataset. Here we'll calculate the great-circle distance between New York and San Francisco.

https://geopy.readthedocs.io/en/stable/#module-geopy.distance  
https://en.wikipedia.org/wiki/Great-circle_distance

In [ ]:
from geopy.distance import great_circle

In [ ]:
# Look up latitude and longitude coorindates with a web search

sanfrancisco = (-122.4, 37.8)  # lon, lat
newyork = (-74.0, 40.7)

In [ ]:
great_circle(sanfrancisco, newyork)

In [ ]:
# This isn't doing a web query or anything; it's just doing math. We can 
# convert the units like this:

great_circle(sanfrancisco, newyork).miles

### Exercise

What's the distance in kilometers between San Francisco and Los Angeles? 

What are the units that are being retured by default?

## 2. Reading geospatial data files

It's easy to read geospatial data files using GeoPandas: shapefiles, GeoJSON, etc.

https://geopandas.org

In [ ]:
import requests

In [ ]:
import geopandas as gpd
print(gpd.__version__)  # note this is a pretty old version

In [ ]:
# download Bay Area storefronts - http://cityobservatory.org/storefront/

url = 'https://github.com/dillonma/storefrontindex/raw/master/all56_nACSxMSA__41860.0.geojson'

with open('all56_nACSxMSA__41860.0.geojson', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [ ]:
# download California census tracts

url = 'https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_06_tract10.zip'

with open('tl_2010_06_tract10.zip', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [ ]:
# Load the storefronts as a GeoDataFrame

storefronts = gpd.read_file('all56_nACSxMSA__41860.0.geojson')

In [ ]:
# Load the census tracts as a GeoDataFrame

tracts = gpd.read_file('zip://tl_2010_06_tract10.zip')  # 'zip://' prefix clarifies format

## 3. Looking at point data

GeoPandas provides interfaces that are very similar to Pandas.

https://geopandas.readthedocs.io/en/v0.3.0/data_structures.html

In [ ]:
storefronts.head()

In [ ]:
len(storefronts)

In [ ]:
# What's the data type?

type(storefronts)

In [ ]:
# Inside, most of the columns are normal Pandas Series

type(storefronts.CITY)

In [ ]:
# But the "geometry" column is special

type(storefronts.geometry)

In [ ]:
# What's each of the items?

one_point = storefronts.geometry[0]

type(one_point)

https://shapely.readthedocs.io/en/latest/manual.html#points

In [ ]:
print(one_point)

In [ ]:
# Shapely points, lines, and polygons have various automatic attributes

print(one_point.x)
print(one_point.y)

In [ ]:
# You can create a Point object from coordinates, too

from shapely.geometry.point import Point

new_point = Point(-122.4, 37.8)

print(new_point)

## 4. Looking at polygon data

In [ ]:
tracts.head(2)

In [ ]:
len(tracts)

In [ ]:
one_polygon = tracts.geometry[0]

type(one_polygon)

In [ ]:
print(one_polygon)

## 5. Performing a spatial join

Having the data in this tidy format lets us do things like perform a *spatial join*, where we calculate which census tract each storefront falls inside.

https://geopandas.org/mergingdata.html#spatial-joins

In [ ]:
# The syntax is just like pd.merge()

merged = gpd.sjoin(storefronts, tracts, how='inner', op='contains')

In [ ]:
# Ok, so the coordinate reference systems don't match. What do we do?

print(storefronts.crs)
print(tracts.crs)

https://epsg.io/4326  
https://epsg.io/4269  
https://www.esri.com/arcgis-blog/products/arcgis-desktop/mapping/wgs84-vs-nad83/

We need to reproject one of the datasets into the coordinate reference system of the other one.

In [ ]:
%%time

storefronts_proj = storefronts.to_crs('epsg:4269')

In [ ]:
# The previous cell takes a long time to run, so we're going to cheat
# and just change the metadata without actually re-projecting the points.
# In this case it's fine because WGS84 and NAD83 are very similar --
# but DON'T DO THIS unless you're sure it's ok!

storefronts_proj = storefronts.copy()
storefronts_proj.crs = tracts.crs

In [ ]:
%%time

merged = gpd.sjoin(storefronts_proj[['CITY','ZIP','geometry']],  # only keep a few columns
                   tracts[['GEOID10','geometry']], 
                   how='inner', 
                   op='intersects')

In [ ]:
merged.head()

In [ ]:
# Check that the number of resulting data points makes sense

len(merged)

## 6. Analyzing the merged data

Now that we have a census tract ID associated with each of the storefronts, we can analyze that information just like in Pandas.

For example, we know that census tracts all have a roughly similar residential population. How much do they vary in number of storefronts?

In [ ]:
merged.groupby('GEOID10').GEOID10.count().describe()

## Exercises

Try loading another spatial dataset into GeoPandas. (If you don't have any handy from your project, you could use the earthquake data from earlier demos.)

What are the data types? Do basic operations like getting descriptive statistics work ok?

Do the coordinates look like lat-lon, or are they other units? What's the coordinate reference system?

Try doing a spatial join. For example, here's some built-in data about countries that you could merge with the earthquakes:

`world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))`